In [2]:

import sys
import os 
import torch
import pandas as pd 
import torchaudio
import IPython.display as ipd
import numpy as np
import random 
from os.path import join 
from torch.utils.data import DataLoader, Dataset
from transformers import Wav2Vec2FeatureExtractor, AutoTokenizer, AutoModel
from torch import nn

sys.path.append("./lib/AngularGrad")
from myoptims.cosangulargrad import cosangulargrad
from sklearn.metrics import f1_score
import pytorch_lightning as pl

os.environ["TOKENIZERS_PARALLELISM"] = "false"

/home/oggy/anaconda3/envs/TorchCuda11.3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0. Process dataset CSV

In [3]:
# data = pd.read_csv('./data/iemocap_2024/iemocap_new_2024.csv')
# data_train = data[data['num_session'].isin([1,2,3,4])]
# data_test  = data[~data['num_session'].isin([1,2,3,4])]

In [4]:
# data_train.to_csv('./data/iemocap_2024/iemocap_train_new_2024.csv', index=False)
# data_test.to_csv('./data/iemocap_2024/iemocap_test_new_2024.csv',index=False)

## 1. Config 

In [5]:
class CFG:
    audio_length=8
    caption_length=50
    resampling_rate=16000
 
    pooling_mode='max' 
    final_dropout=0.2
    model_name='Hubert_DistilBert_CLIP'

    feature_speech=768 
    feature_text = 768
    embedding_size = 512

    temperature=1.0
    learn_rate_speech=1e-4
    learn_rate_text=1e-5 
    min_lr=1e-6

    train_batch_size=32
    train_num_workers=4
    val_batch_size=32
    val_num_workers=4
        
    csv_train_path='./data/iemocap_2024/iemocap_new_2024.csv'
    csv_valid_path='./data/iemocap_2024/iemocap_new_2024.csv' 
    
    speech_pretrained_path='./pretrained/hubert-base-ls960'
    text_pretrained_path='./pretrained/distilbert-base-uncased'


## 1. Dataset Loader 

In [6]:
def Convert2Mono(audio_tensor ,channel_first: bool =True):
    '''
    Convert Stereo to Mono 
    Stereo: audio xuất hiện 2 channels (tượng trưng cho left-right sound)
    Mono: audio chỉ có 1 channel 
    Ref: https://github.com/pytorch/audio/issues/877#issuecomment-673472645
    '''

    # 1D array --> Adding channel num 
    if len(audio_tensor.shape)==1:
        return audio_tensor.unsqueeze(0)

    # 2D array
    elif len(audio_tensor.shape)==2:
        ## Stereo audio 
        if int(audio_tensor[0]) == 2:
            output = torch.mean(audio_tensor, dim=0) if channel_first else torch.mean(audio_tensor, dim=1)
        ## Mono Audio 
        else:
            output = audio_tensor
        return output
    

def truncate_audio(audio_tensor, 
                   sampling_rate: int, 
                   max_time: int):
    '''
    Resize to the same length (padding or truncate)
    Ref: https://tiensu.github.io/blog/70_audio_deep_learning_part_4/
    Thay đổi chiều dài của tất cả các files Audio về chung một giá trị max_length. 
    Audio < max_length: sẽ được kéo dài bằng cách thêm vào khoảng im lặng - slience. 
    Audio > max_length: cắt bớt đi.

    audio_array: [Tensor]:  
    sampling_rate [int] 
    max_time: [int]
    '''

    num_rows, audio_len = audio_tensor.shape # channel_num, frame_num
    max_len = sampling_rate*max_time
    
    # Cut audio 
    if (audio_len > max_len):
        audio_tensor = audio_tensor[:,:max_len]

    # Padding to add at the beginning and end of the signal
    elif (audio_len < max_len):
        pad_begin_len = random.randint(0, max_len - audio_len)
        pad_end_len = max_len - audio_len - pad_begin_len

        # Pad with 0-values 
        pad_begin = torch.zeros((num_rows, pad_begin_len))
        pad_end = torch.zeros((num_rows, pad_end_len))
        audio_tensor = torch.cat((pad_begin, audio_tensor, pad_end), 1)

    return audio_tensor


def get_audio_path(file_name):
    return join('./data/iemocap_2024/iemocap/',file_name+'.wav')


def get_class_num(class_name):
    if class_name == 'neu':
        return 0 
    elif class_name == 'sad':
        return 1
    elif class_name == 'ang':
        return 2
    elif class_name == 'exc' or class_name == 'hap':
        return 3 
    else:
        print('~~~~~~~~~~~~ Fuck ~~~~~~~~~~~')
        return None

In [7]:
class Hubert_Text_Classification(Dataset):
    def __init__(self, 
            csv_path: str, 
            audio_length: int = None,
            caption_length: int = None,
            resampling_rate: int = None,
            text_pretrained_path=None,
            speech_pretrained_path: str = None
    ) -> None:
        super().__init__()
        
        self.df = pd.read_csv(csv_path)
        self.df['audio_paths']  = self.df['FileName'].apply(get_audio_path)
        self.df['class_labels'] = self.df['Label'].apply(get_class_num) 

        self.audio_paths = list(self.df['audio_paths'])
        self.captions= list(self.df['Sentences'])
        self.labels  = list(self.df['class_labels'])

        self.audio_length = audio_length
        self.resampling_rate = resampling_rate
        self.tokenizer = AutoTokenizer.from_pretrained(
            text_pretrained_path,
            local_files_only=True
        )
        self.tokenized_captions = self.tokenizer(
            self.captions,
            padding=True, 
            truncation=True, 
            max_length=caption_length,
            return_tensors='pt'
        )
        self.hubert_processor = Wav2Vec2FeatureExtractor.from_pretrained(
            speech_pretrained_path, 
            local_files_only=True
        )
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # 0. Read the audio 
        speech, sampling_rate = torchaudio.load(self.audio_paths[idx])
        
        # 1. Resample Audio 
        resampler = torchaudio.transforms.Resample(sampling_rate, self.resampling_rate)
        speech = resampler(speech).squeeze()

        # 2. Convert Stero2Mono 
        speech = Convert2Mono(speech, channel_first=True) 
        
        # 3. Resize make sure all audio same length  
        speech = truncate_audio(
            audio_tensor=speech, 
            sampling_rate=self.resampling_rate, 
            max_time=self.audio_length
        )

        # 4. Wave2Vec process audio --> return audio array (channel, signal) [Tensor]
        speech = self.hubert_processor(
            raw_speech=speech,
            padding=False,
            max_length=None,
            truncation=False,
            return_tensors='pt',
            sampling_rate=self.resampling_rate,
        ).input_values[0]
        
        # 5. Text 
        text_input_ids = self.tokenized_captions['input_ids'][idx] 
        text_att_mask  = self.tokenized_captions['attention_mask'][idx]
        
        # 6. Label  
        d_type = torch.long if isinstance(self.labels[idx], int) else torch.float
        label = torch.tensor(self.labels[idx], dtype=d_type)
        
        return {'audio': speech, 
                'label': label, 
                'input_ids': text_input_ids, 
                'attention_mask': text_att_mask}

## 2. Lightning Dataloader 

In [8]:
import logging
from pytorch_lightning import LightningDataModule, LightningModule

class Hubert_Text_Dataset(LightningDataModule):
    def __init__(self, csv_train_path, csv_valid_path):
        super().__init__()
        self.csv_train_path = csv_train_path
        self.csv_valid_path = csv_valid_path
        self.dataloader = Hubert_Text_Classification

    def prepare_data(self):
        pass 
    @property
    def num_classes(self) -> int:
        return int(self.train_csv[''].nunique())   
    
    def setup(self, *_, **__) -> None:
        self.train_dataset = self.dataloader(
            csv_path=CFG.csv_train_path, 
            audio_length=CFG.audio_length,
            caption_length=CFG.caption_length,
            resampling_rate=CFG.resampling_rate,
            text_pretrained_path=CFG.text_pretrained_path,
            speech_pretrained_path=CFG.speech_pretrained_path
        )
        logging.info(f"training dataset: {len(self.train_dataset)}")
        
        self.val_dataset = self.dataloader(
            csv_path=CFG.csv_valid_path, 
            audio_length=CFG.audio_length,
            caption_length=CFG.caption_length,
            resampling_rate=CFG.resampling_rate,
            text_pretrained_path=CFG.text_pretrained_path,
            speech_pretrained_path=CFG.speech_pretrained_path
        )
        logging.info(f"validation dataset: {len(self.val_dataset)}")


    def train_dataloader(self) -> DataLoader:
        return DataLoader(
                self.train_dataset,
                batch_size=CFG.train_batch_size ,
                num_workers=CFG.train_num_workers,
                pin_memory=True,
                drop_last=True, 
                shuffle=True
            )
    
    def val_dataloader(self) -> DataLoader:
        return DataLoader(
                self.val_dataset,
                batch_size=CFG.val_batch_size ,
                num_workers=CFG.val_num_workers,
                pin_memory=True,
                drop_last=False,
                shuffle=False
            )


### 2. Projection Head 

In [9]:
class ProjectionHead(nn.Module):
    def __init__(self, embedding_dim: int, projection_dim: int, dropout: float) -> None:
        super().__init__()

        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x += projected
        return self.layer_norm(x)


### 3. Text Encoder 

In [10]:
class TextEncoder(nn.Module):
    def __init__(self, pretrained_path: str, trainable: bool = True) -> None:
        super().__init__()

        self.model = AutoModel.from_pretrained(pretrained_path)
        self.hidden_size = self.model.transformer.layer[5].output_layer_norm.normalized_shape[0]
        self.target_token_idx = 0
        self.project = ProjectionHead(
            embedding_dim=CFG.feature_text,
            projection_dim=CFG.embedding_size,
            dropout=0.2,
        )

        for param in self.model.parameters():
            param.requires_grad = trainable
        
    # Feature size=768
    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        text_embedding = self.project(last_hidden_state[:, self.target_token_idx, :])
        return text_embedding

### 4.Speech Encoder 

In [11]:
from transformers import AutoConfig

class HubertForSpeechEncoder(nn.Module):
    def __init__(self, pooling_mode, pretrained_path):
        super().__init__()
        
        self.pooling_mode = pooling_mode
        self.hubert = AutoModel.from_pretrained(pretrained_path)
        self.project = ProjectionHead(
            embedding_dim=CFG.feature_speech,
            projection_dim=CFG.embedding_size,
            dropout=0.2,
        )
        
    def freeze_feature_extractor(self):
        self.hubert.feature_extractor._freeze_parameters()

    def merged_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")
        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
    ):  
        input_values = input_values.squeeze(1)
        outputs = self.hubert(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        speech_embedding = self.project(hidden_states)
        return speech_embedding

### 5. Base Training Loop

In [12]:
import torch.nn.functional as F 

class BaseTraining(LightningModule):
    def __init__(self):
        super().__init__()
        self.speech_model = HubertForSpeechEncoder(
            pooling_mode=CFG.pooling_mode, 
            pretrained_path=CFG.speech_pretrained_path
        )
        self.text_model =  TextEncoder(
            pretrained_path=CFG.text_pretrained_path, 
            trainable=True
        )

        self.learn_rate_speech = CFG.learn_rate_speech
        self.learn_rate_text = CFG.learn_rate_text
        self.temperature = CFG.temperature
        self.loss = nn.LogSoftmax(dim=-1)
        self.log_softmax = nn.LogSoftmax(dim=-1)
    
    def forward(self, batch):
        speech_emb = self.speech_model(batch['audio'])
        text_emb = self.text_model(batch['input_ids'], batch['attention_mask'])
        return speech_emb, text_emb

    def compute_losses(self, speech_embeddings, text_embeddings):
        logits = (text_embeddings @ speech_embeddings.T) / self.temperature
        images_similarity = speech_embeddings @ speech_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax(
            (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
        )
        images_loss = (-targets.T * self.log_softmax(logits.T)).sum(1)
        texts_loss = (-targets * self.log_softmax(logits)).sum(1)
        return (images_loss + texts_loss) / 2.0


    def training_step(self, batch, batch_idx):
        speech_emb, text_emb = self.forward(batch)
        loss = self.compute_losses(speech_emb, text_emb).mean()
        self.log("loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        speech_emb, text_emb = self.forward(batch)
        loss = self.compute_losses(speech_emb, text_emb).mean()
        return {"loss": loss.to('cpu').item()}

    def validation_epoch_end(self, outputs):
        loss_list = [] 
        for out in outputs:
            loss_list.append(out['loss'])
        loss  = np.array(loss_list).mean()

        self.log('loss_val',loss,sync_dist=True,on_epoch=True,prog_bar=True)

        
    def configure_optimizers(self):
        parameters = [
            {"params": self.speech_model.parameters(), "lr": self.learn_rate_speech},
            {"params": self.text_model.parameters(), "lr": self.learn_rate_text},
        ]

        optimizer = cosangulargrad(parameters)  
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, 
            self.trainer.max_epochs, 
            CFG.min_lr
        ) 
        return [optimizer], [scheduler]
    
    # def Custom_save_model_pretrained(self,PATH):
    #     torch.save(self.model.state_dict(), os.path.join(PATH) + f"/model_best_{self.model.arch}.pth")
    #     print(f"Saved: model_best_{self.model.arch}.pth")


## 7. Training Loop

In [15]:
DataModule = Hubert_Text_Dataset(
    csv_train_path=CFG.csv_train_path,
    csv_valid_path=CFG.csv_valid_path
)
DataModule.setup()

In [16]:
model = BaseTraining()

In [17]:
# Logging
logger_name = f'{CFG.model_name}_LenAudio{CFG.audio_length}' 
logger = pl.loggers.CSVLogger(save_dir='./Output/', name=logger_name)
print("---->",logger_name)

# Checkpoint saving
ckpt = pl.callbacks.ModelCheckpoint(
    monitor='loss_val',
    save_top_k=1,
    save_last=True,     
    save_weights_only=True,
    filename='{epoch:02d}-{loss_val:.3f}',
    mode='min'
)

# Trainer 
trainer = pl.Trainer(
    accelerator='gpu',
    move_metrics_to_cpu=False,
    gpus=[0],
    callbacks=[ckpt],
    logger=logger,
    max_epochs=50,
    precision='bf16', # 16bit autocast 
    accumulate_grad_batches=1,
    check_val_every_n_epoch=1,
    # resume_from_checkpoint=''
)

# Training
trainer.fit(model=model, datamodule=DataModule)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/oggy/anaconda3/envs/TorchCuda11.3/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


----> Hubert_DistilBert_CLIP_LenAudio8



  | Name         | Type                   | Params
--------------------------------------------------------
0 | speech_model | HubertForSpeechEncoder | 95.0 M
1 | text_model   | TextEncoder            | 67.0 M
2 | loss         | LogSoftmax             | 0     
3 | log_softmax  | LogSoftmax             | 0     
--------------------------------------------------------
162 M     Trainable params
0         Non-trainable params
162 M     Total params
648.198   Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

2024-02-25 04:07:24.703577: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 04:07:24.703624: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 04:07:24.704385: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 04:07:24.704426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 04:07:24.704727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory

...

Epoch 49:   0%|          | 0/345 [00:00<?, ?it/s, loss=0.192, v_num=0, loss_val=0.119]          

2024-02-25 06:18:27.050305: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 06:18:27.050349: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 06:18:27.051411: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 06:18:27.052014: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 06:18:27.052070: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factor

Epoch 49:  50%|████▉     | 172/345 [02:08<02:09,  1.33it/s, loss=0.0202, v_num=0, loss_val=0.119] 

2024-02-25 06:20:36.095831: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 06:20:36.095880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 06:20:36.096395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 06:20:36.096446: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 06:20:36.097003: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory

Epoch 49: 100%|██████████| 345/345 [02:40<00:00,  2.14it/s, loss=0.0202, v_num=0, loss_val=0.125]


## 8. Saving Model 

In [15]:
class BaseSaving(LightningModule):
    def __init__(self):
        super().__init__()
        self.speech_model = HubertForSpeechEncoder(
            pooling_mode=CFG.pooling_mode, 
            pretrained_path=CFG.speech_pretrained_path
        )
        self.text_model =  TextEncoder(
            pretrained_path=CFG.text_pretrained_path, 
            trainable=True
        )

    def save_speech_weight(self,PATH, name_file):
        torch.save(self.speech_model.state_dict(), os.path.join(PATH, name_file))
        print(f'----> Saved: {os.path.join(PATH, name_file)}')

In [16]:
model = BaseSaving()
weight = torch.load('/home/oggy/Desktop/AI_Sound_2024/Output/Hubert_DistilBert_CLIP_LenAudio8/version_0/checkpoints/epoch=48-loss_val=0.119.ckpt')
model.load_state_dict(weight['state_dict'])

<All keys matched successfully>

In [17]:
model.save_speech_weight('./Output', 'Hubert_DistilBet_CLIP_0.119_8s.pth')

----> Saved: ./Output/Hubert_DistilBet_CLIP_0.119_8s.pth
